In [1]:
%load_ext autoreload
%autoreload 2

import argparse
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '8'
os.chdir("..")

import math
import yaml
import logging
import random
import numpy as np
import sys
import imageio
import torch

os.getcwd()

'/nfs/horai.dgpsrv/ondemand28/harryscz/diffusion'

In [2]:
def parse_args(arg_list=None):
    parser = argparse.ArgumentParser(
        description="Unconditioned Video Diffusion Inference"
    )
    parser.add_argument(
        "--dataset-path", type=str, required=True,
        help="Directory containing input reference videos."
    )
    parser.add_argument(
        "--pretrained-model-name-or-path", type=str, required=True,
        help="Path or HF ID where transformer/vae/scheduler are stored."
    )
    parser.add_argument(
        "--checkpoint-path", type=str, required=True,
        help="Path to fine‐tuned checkpoint containing transformer state_dict."
    )
    parser.add_argument(
        "--output-dir", type=str, required=True,
        help="Where to write generated videos."
    )
    parser.add_argument(
        "--model-config", type=str, required=True,
        help="YAML file describing model params (height, width, num_reference, num_target, etc.)"
    )
    parser.add_argument(
        "--batch-size", type=int, default=1,
        help="Batch size per device (usually 1 for inference)."
    )
    parser.add_argument(
        "--num-inference-steps", type=int, default=50,
        help="Number of reverse diffusion steps to run."
    )
    parser.add_argument(
        "--mixed-precision", type=str, default="bf16",
        help="Whether to run backbone in 'fp16', 'bf16', or 'fp32'."
    )
    parser.add_argument(
        "--seed", type=int, default=42,
        help="Random seed for reproducibility."
    )
    parser.add_argument(
        "--shuffle", type=int, default=False,
        help="Whether to shuffle dataset. Usually False for inference."
    )
    parser.add_argument(
        "--is-uncond", type=bool, default=False,
        help=""
    )
    parser.add_argument(
        "--sample-frames", type=int, default=50
    )

    # If arg_list is None, argparse picks up sys.argv; 
    # otherwise it treats arg_list as the full argv list.
    return parser.parse_args(arg_list)

args = [
    "--dataset-path", "/scratch/ondemand28/harryscz/head_audio/head/data/vfhq-fit",
    "--pretrained-model-name-or-path", "/scratch/ondemand28/harryscz/model/CogVideoX-2b",
    "--checkpoint-path",  "/scratch/ondemand28/harryscz/head_audio/trainOutput/checkpoint-6000.pt",
    "--output-dir",  "/scratch/ondemand28/harryscz/diffusion/videoOut",
    "--model-config",  "/scratch/ondemand28/harryscz/diffusion/train/model_config.yaml",
    "--batch-size",  "1",
    "--num-inference-steps",  "50",
    "--mixed-precision",  "no",
    "--seed",  "42",
    "--shuffle",  "0",
    "--sample-frames", "29"
]

args = parse_args(args)

with open(args.model_config, "r") as f: model_config = yaml.safe_load(f)


In [3]:
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs, ProjectConfiguration, set_seed
from accelerate.logging import get_logger

with open(args.model_config, "r") as f: model_config = yaml.safe_load(f)
if args.mixed_precision.lower() == "fp16":
    dtype = torch.float16
elif args.mixed_precision.lower() == "bf16":
    dtype = torch.bfloat16
else:
    dtype = torch.float32

accelerator_project_config = ProjectConfiguration(project_dir=args.output_dir,
                                                    logging_dir=os.path.join(args.output_dir, "logs"))
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=False)
accelerator = Accelerator(mixed_precision=args.mixed_precision,
                            project_config=accelerator_project_config,
                            kwargs_handlers=[ddp_kwargs])

# 2.4 Set random seed
if args.seed is not None:
    set_seed(args.seed + accelerator.process_index)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

logger = get_logger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info("Accelerator state:", accelerator.state)

--- Logging error ---
Traceback (most recent call last):
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/runpy.py", line 87, in

In [ ]:
#### Dataset #####
# Video data have shape [B, C, F, H, W]

%load_ext autoreload
%autoreload 2

from data.VideoDataset import *
from torch.utils.data import DataLoader, DistributedSampler

dataset = VideoPathDataset(
    source_dir=args.dataset_path,
)
if args.shuffle:
    sampler = DistributedSampler(
        dataset,
        num_replicas=accelerator.num_processes,
        rank=accelerator.process_index,
        shuffle=True
    )
else:
    sampler = None
data_loader = DataLoader(
    dataset,
    batch_size=1,
    # sampler=sampler,
    collate_fn=lambda x: x,   
    num_workers=2,
    pin_memory=True,
)
logger.info(f"Number of test examples: {len(data_loader)}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


07/21/2025 10:37:56 - INFO - __main__ - Number of test examples: 1


In [5]:
from model.flameObj import *

flamePath = "/scratch/ondemand28/harryscz/head_audio/head/code/flame/flame2023_no_jaw.npz"
sourcePath = "/scratch/ondemand28/harryscz/head_audio/head/data/vfhq-fit"
dataPath = [os.path.join(os.path.join(sourcePath, data), "fit.npz") for data in os.listdir(sourcePath)]
seqPath = "/scratch/ondemand28/harryscz/head/_-91nXXjrVo_00/fit.npz"

head = Flame(flamePath, device="cuda")

# from tqdm import tqdm_notebook as tqdm

# for i in tqdm(range(0, len(dataPath), 2)):
#     try:
#         x = head.batch_uv(dataPath[i:i+2], sample_frames=29)
#         assert x.shape[1] == 29
#     except:
#         print(dataPath[i:i+2])

/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [6]:
def encode_video(vae, video):
    video = video.to(accelerator.device, dtype=vae.dtype)
    video = video.permute(0, 2, 1, 3, 4)  # [B, C, F, H, W]
    with torch.no_grad(): latent_dist = vae.encode(video).latent_dist.sample() * vae.config.scaling_factor
    return latent_dist.permute(0, 2, 1, 3, 4).to(memory_format=torch.contiguous_format)

In [7]:
#### Load Model ####
device = "cuda"
dtype = torch.float32

from diffusers import AutoencoderKLCogVideoX, CogVideoXDPMScheduler
from model.cap_transformer import CAPVideoXTransformer3DModel

transformer = CAPVideoXTransformer3DModel.from_pretrained(
    args.pretrained_model_name_or_path,
    low_cpu_mem_usage=False,
    device_map=None,
    ignore_mismatched_sizes=True,
    subfolder="transformer",
    torch_dtype=torch.float32,
    cond_in_channels=16,  # only one channel (the ref_mask)
    sample_width=model_config["width"] // 8,
    sample_height=model_config["height"] // 8,
    sample_frames=args.sample_frames,
    max_text_seq_length=1,
    max_n_references=model_config["max_n_references"],
    apply_attention_scaling=model_config["use_growth_scaling"],
    use_rotary_positional_embeddings=False,
)
vae = AutoencoderKLCogVideoX.from_pretrained(
    args.pretrained_model_name_or_path, subfolder="vae"
)
scheduler = CogVideoXDPMScheduler.from_pretrained(
    args.pretrained_model_name_or_path, subfolder="scheduler"
)

vae.eval().to(dtype)
transformer.eval().to(dtype)


Some weights of the model checkpoint at /scratch/ondemand28/harryscz/model/CogVideoX-2b were not used when initializing CAPVideoXTransformer3DModel: 
 ['patch_embed.text_proj.weight, patch_embed.text_proj.bias']
Some weights of CAPVideoXTransformer3DModel were not initialized from the model checkpoint at /scratch/ondemand28/harryscz/model/CogVideoX-2b and are newly initialized: ['patch_embed.ref_temp_proj.bias', 'patch_embed.audio_proj.weight', 'patch_embed.ref_temp_proj.weight', 'patch_embed.cond_proj.bias', 'patch_embed.audio_proj.bias', 'patch_embed.cond_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CAPVideoXTransformer3DModel(
  (patch_embed): CAPPatchEmbed(
    (proj): Conv2d(16, 1920, kernel_size=(2, 2), stride=(2, 2))
    (cond_proj): Conv2d(16, 1920, kernel_size=(2, 2), stride=(2, 2))
    (audio_proj): Linear(in_features=3072, out_features=1920, bias=True)
    (ref_temp_proj): Linear(in_features=2, out_features=480, bias=True)
  )
  (embedding_dropout): Dropout(p=0.0, inplace=False)
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=1920, out_features=512, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=512, out_features=512, bias=True)
  )
  (transformer_blocks): ModuleList(
    (0-29): 30 x CogVideoXBlock(
      (norm1): CogVideoXLayerNormZero(
        (silu): SiLU()
        (linear): Linear(in_features=512, out_features=11520, bias=True)
        (norm): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
      )
      (attn1): Attention(
        (norm_q): LayerNorm((64,), eps=1e-06, elementwise_affine=Tru

In [8]:
ckpt_path = "/scratch/ondemand28/harryscz/diffusion/modelOut/checkpoint-16000.pt"
ckpt = torch.load(ckpt_path, map_location="cpu")
transformer.load_state_dict(ckpt["state_dict"], strict=False)

vae, transformer, scheduler, data_loader = accelerator.prepare(vae, transformer, scheduler, data_loader)

/tmp/ipykernel_282910/3094665471.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location="cpu")


In [9]:
vae, transformer, scheduler, data_loader = accelerator.prepare(vae, transformer, scheduler, data_loader)

In [8]:


from inference._inference_pipeline import *

pipe = CAPVideoPipeline(
    vae=vae,
    transformer=transformer,
    scheduler=scheduler,
)

In [10]:
data = iter(data_loader)
batch = next(data)

In [11]:
batch

['/scratch/ondemand28/harryscz/head_audio/head/data/vfhq-fit/g1eIAelVFq4_02/fit.npz']

In [12]:
latent_chunks = []
uncond_latent_chunks = []
cond_chunks = []
uncond_chunks = []
cond_vises = []
ref_mask_chunks = []

uvs = head.batch_uv(batch, resolution=256, rotation=False, sample_frames=args.sample_frames).permute(0,1,4,2,3) # load UVs of shape B, F, C, H, W
ref_frame = uvs[:, 0, :, :, :].unsqueeze(1) # B, 1, C, H, W

latent_chunk = encode_video(vae, uvs).to(dtype=dtype)
_ref_latent = encode_video(vae, ref_frame).to(dtype=dtype)
ref_latent = torch.zeros(latent_chunk.shape).to(dtype=dtype, device=accelerator.device)
ref_latent[:, 0, ...] = _ref_latent.squeeze(1)
ref_mask = torch.zeros(latent_chunk.shape).to(accelerator.device)
ref_mask[:, 0, ...] = 1.0

# ref_mask_chunks.append(cond_chunk["ref_mask"].permute(0, 1, 4, 2, 3))
# cond_vis = conditioning.get_vis(cond_chunk["condition"])
# cond_chunk = einops.rearrange(cond_chunk["condition"], 'b f h w c -> b f c h w').to(device=device)
# cond_chunk = cond_chunk.to(dtype=dtype)
latent_chunks.append(latent_chunk)
uncond_latent_chunks.append(latent_chunk * 0.)
cond_chunks.append(ref_latent)
# cond_vises.append(cond_vis)
uncond_chunks.append(ref_latent * 0.)
ref_mask_chunks.append(ref_mask)
B, F_z, C, H_z, W_z = latent_chunks[0].shape
    
# with torch.no_grad():
#     audio_encoding = encode_audio(
#         audio_model=audio_model,
#         audio=batch["audio"].to(dtype=dtype, device=device),
#         sample_positions=batch["sample_positions"].to(dtype=dtype, device=device),
#     )
# if not batch["has_audio"][0]:  # if there is no audio, set audio encoding to zero
#     audio_encoding = audio_encoding * 0.
audio_encoding = torch.zeros(
    (B, 3, 768), dtype=torch.float32, device=accelerator.device
)
uncond_audio_encoding = audio_encoding * 0.
text_embeds = torch.zeros(1, 1, 1920, device=device, dtype=dtype)

sequence_infos = []
for chunk_id, latent in enumerate(latent_chunks):
    sequence_infos.append((False, torch.arange(0, latent.shape[1], device=accelerator.device)))

out = pipe(
    height=256,
    width=256,
    num_frames=29,
    num_inference_steps=50,
    conditioning=cond_chunks,
    uncond_conditioning=uncond_chunks,
    latents=latent_chunks,
    uncond_latents=uncond_latent_chunks,
    ref_mask_chunks=ref_mask_chunks,
    audio_embeds=audio_encoding,
    uncond_audio_embeds=uncond_audio_encoding,
    text_embeds=text_embeds,
    uncond_text_embeds=text_embeds,
    sequence_infos=sequence_infos, 
    output_type="pt",
)

/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


  0%|          | 0/50 [00:00<?, ?it/s]

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
i = 0

uvs = head.batch_uv(dataPath[:1], resolution=256, rotation=False, sample_frames=args.sample_frames).permute(0,1,4,2,3) # load UVs of shape B, F, C, H, W
orig = uvs[i].permute(0,2,3,1)

recon = out[0][0][0].permute(0,2,3,1)

oUV = head.sampleFromUV(orig, savePath=f"diffOut/{i}_uv.mp4") # input has shape F, H, W, C
head.sampleTo3D(oUV, savePath=f"diffOut/{i}_3d.mp4", resolution=512, dist=1.2, azim=20, elev=10)

diffUV = head.sampleFromUV(recon, savePath=f"diffOut/{i}_diff_uv.mp4") # input has shape F, H, W, C
head.sampleTo3D(diffUV, savePath=f"diffOut/{i}_diff_3d.mp4", resolution=512, dist=1.2, azim=20, elev=10)

/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


tensor([[[ 0.2242,  0.0974, -2.5384],
         [ 0.2242,  0.0974, -2.5384],
         [ 0.2236,  0.0980, -2.5376],
         ...,
         [ 0.1003,  0.1259, -2.4684],
         [ 0.1003,  0.1259, -2.4684],
         [ 0.1003,  0.1259, -2.4684]],

        [[ 0.2242,  0.0974, -2.5384],
         [ 0.2242,  0.0974, -2.5384],
         [ 0.2236,  0.0980, -2.5376],
         ...,
         [ 0.1003,  0.1259, -2.4684],
         [ 0.1003,  0.1259, -2.4684],
         [ 0.1003,  0.1259, -2.4684]],

        [[ 0.2242,  0.0974, -2.5384],
         [ 0.2242,  0.0974, -2.5384],
         [ 0.2236,  0.0980, -2.5376],
         ...,
         [ 0.1003,  0.1259, -2.4684],
         [ 0.1003,  0.1259, -2.4684],
         [ 0.1003,  0.1259, -2.4684]],

        ...,

        [[ 0.2284,  0.0986, -2.5359],
         [ 0.2284,  0.0986, -2.5359],
         [ 0.2276,  0.0989, -2.5356],
         ...,
         [ 0.1043,  0.1250, -2.4653],
         [ 0.1043,  0.1250, -2.4653],
         [ 0.1043,  0.1250, -2.4653]],

        [[

In [20]:
def decode_latents(vae, latents: torch.Tensor) -> torch.Tensor:
    latents = latents.permute(0, 2, 1, 3, 4)  # [batch_size, num_channels, num_frames, height, width]
    latents = 1 / vae.config.scaling_factor * latents

    with torch.no_grad(): frames = vae.decode(latents).sample
    return frames
i = 0
uvs = head.batch_uv(dataPath[:1], resolution=256, rotation=False, sample_frames=args.sample_frames).permute(0,1,4,2,3) # load UVs of shape B, F, C, H, W
z = encode_video(vae, uvs)
y = decode_latents(vae,z)[0].permute(1,2,3,0)

diffUV = head.sampleFromUV(y, savePath=f"diffOut/{i}_vae_uv.mp4") # input has shape F, H, W, C
head.sampleTo3D(diffUV, savePath=f"diffOut/{i}_vae_3d.mp4", resolution=512, dist=1.2, azim=20, elev=10)

/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


tensor([[[-0.0878, -0.2992, -3.0141],
         [-0.0878, -0.2992, -3.0141],
         [-0.0893, -0.2974, -3.0125],
         ...,
         [-0.3341, -0.2423, -2.8698],
         [-0.3341, -0.2423, -2.8698],
         [-0.3341, -0.2423, -2.8698]],

        [[-0.0878, -0.2992, -3.0141],
         [-0.0878, -0.2992, -3.0141],
         [-0.0893, -0.2974, -3.0125],
         ...,
         [-0.3341, -0.2423, -2.8698],
         [-0.3341, -0.2423, -2.8698],
         [-0.3341, -0.2423, -2.8698]],

        [[-0.0878, -0.2992, -3.0141],
         [-0.0878, -0.2992, -3.0141],
         [-0.0893, -0.2974, -3.0125],
         ...,
         [-0.3341, -0.2423, -2.8698],
         [-0.3341, -0.2423, -2.8698],
         [-0.3341, -0.2423, -2.8698]],

        ...,

        [[-0.0845, -0.2977, -3.0104],
         [-0.0845, -0.2977, -3.0104],
         [-0.0854, -0.2955, -3.0094],
         ...,
         [-0.3335, -0.2420, -2.8668],
         [-0.3335, -0.2420, -2.8668],
         [-0.3335, -0.2420, -2.8668]],

        [[

In [19]:
y.shape

torch.Size([32, 256, 256, 3])

In [ ]:
i = 1
head.loadSequence(dataPath[i])
origMesh = head.LSB(rotation=False)
# head.renderVertsAnimation(savePath=f"flameOut/{i}orig.mp4", resolution=512)
uvMesh = head.convertUV(rotation=False)
uvs = head.get_uv_animation(uvMesh, savePath=f"flameOut/{i}uv.mp4", resolution=512)
sampledUV = head.sampleFromUV(uvs, savePath=f"flameOut/{i}sample.mp4", resolution=512)
head.sampleTo3D(sampledUV, savePath=f"flameOut/{i}recon.mp4", resolution=512, dist=1.2)

tensor([[[-0.0865, -0.2964, -3.0184],
         [-0.0858, -0.2966, -3.0159],
         [-0.0858, -0.2954, -3.0154],
         ...,
         [-0.3338, -0.2333, -2.8628],
         [-0.3301, -0.2330, -2.8595],
         [-0.3301, -0.2330, -2.8595]],

        [[-0.0866, -0.2964, -3.0185],
         [-0.0860, -0.2966, -3.0160],
         [-0.0860, -0.2954, -3.0154],
         ...,
         [-0.3339, -0.2332, -2.8628],
         [-0.3302, -0.2329, -2.8594],
         [-0.3302, -0.2329, -2.8594]],

        [[-0.0867, -0.2963, -3.0185],
         [-0.0861, -0.2966, -3.0160],
         [-0.0861, -0.2954, -3.0155],
         ...,
         [-0.3342, -0.2333, -2.8629],
         [-0.3305, -0.2331, -2.8596],
         [-0.3305, -0.2331, -2.8596]],

        ...,

        [[-0.0866, -0.2960, -3.0177],
         [-0.0859, -0.2962, -3.0152],
         [-0.0859, -0.2950, -3.0146],
         ...,
         [-0.3328, -0.2328, -2.8617],
         [-0.3291, -0.2325, -2.8584],
         [-0.3291, -0.2325, -2.8584]],

        [[

: 

In [20]:
out[0][0][0].permute(1,0,2,3).shape

torch.Size([3, 32, 512, 512])

In [72]:
def save_video(video_tensor, path):
    video_tensor = video_tensor.squeeze(0)  # [3, 49, 256, 256]
    video_tensor = video_tensor.permute(1, 2, 3, 0)  # [49, 256, 256, 3]

    # Normalize to 0–255 and convert to uint8 if needed
    video_np = (video_tensor * 255).clamp(0, 255).byte().cpu().numpy()

    # Save as .mp4
    imageio.mimsave(path, video_np, fps=16)

save_video(out[0][0][0].permute(1,0,2,3) , "/scratch/ondemand28/harryscz/diffusion/videoOut/ref1kUV3.mp4")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [ ]:
head.loadSequence(dataPath[1])                                       # PASS
perFrameVerts = head.LSB()                                           # PASS
# head.renderVertsAnimation("cache/original3d.mp4", customVerts=perFrameVerts)                         # PASS
uvMesh = head.convertUV(rotation=False)                                            # CHECKED
perFrameUV = head.get_uv_animation(uvMesh, "cache/originalUV.mp4", fill=True, resolution=512)           # ___
sampledUV = head.sampleFromUV(perFrameUV, "cache/sampledUV.mp4", resolution=512)     # ___
sampled3dSeq = head.sampleTo3D(sampledUV, "cache/sampled3d.mp4", dist=1.5) 

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


NameError: name 'z' is not defined

In [74]:
uv = out[0][0][0].permute(0,2,3,1)
sampledUV = head.sampleFromUV(uv, "videoOut/ref1kUV3.mp4", resolution=256) # Input need to be N, W, H, C
sampled3dSeq = head.sampleTo3D(sampledUV, "videoOut/ref1kUV33d.mp4", dist=1, scale=1.6)  

In [ ]:
i = 0

vid_dir = os.listdir("/scratch/ondemand28/harryscz/head_audio/data/data256/uv/")
vid_path = os.path.join("/scratch/ondemand28/harryscz/head_audio/data/data256/uv/", vid_dir[i])

video, audio, info = torchvision.io.read_video(vid_path, pts_unit='sec')
video = (video.float() / 255).to("cuda")[:29 , ...]

frames_rgb = (video * 255).byte().cpu().numpy()  
imageio.mimsave(f"videoOut/{i}_OrigUV.mp4", frames_rgb, fps=16, quality=10) 

sampledUV = head.sampleFromUV(video, f"videoOut/{i}_SampledUV.mp4", resolution=256) # Input need to be N, W, H, C
sampled3dSeq = head.sampleTo3D(sampledUV, f"videoOut/{i}_recon3d.mp4", dist=1, scale=1.6)  

In [ ]:
head.loadSequence(dataPath[20])
head.LSB()
uvMesh = head.convertUV()
uv = head.get_uv_animation(uvMesh, savePath=f"videoOut/20_uv.mp4", resolution=256)
torch.save(uv, "videoOut/20.pt")


/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [7]:
data = iter(data_loader)
next(data)

NameError: Caught NameError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/nfs/horai.dgpsrv/ondemand28/harryscz/diffusion/data/VideoDataset.py", line 55, in __getitem__
    head.loadSequence(path)
NameError: name 'head' is not defined


In [126]:
uv = torch.load("videoOut/20.pt")
sampledUV = head.sampleFromUV(uv, "videoOut/20uv.mp4", resolution=256) # Input need to be N, W, H, C
sampled3dSeq = head.sampleTo3D(sampledUV, "videoOut/203d.mp4", dist=1, scale=1.6)  

/tmp/ipykernel_2226707/2551400825.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  uv = torch.load("videoOut/20.pt")
